## Import libraries

In [ ]:
import pandas as pd
from nltk.corpus import stopwords as s 
import re
from nltk.tokenize import word_tokenize  
from scipy.stats import chi2_contingency 
from nltk.stem import WordNetLemmatizer
import keras
import tensorflow as tf

## Data exploration and cleaning

In [2]:
data=pd.read_csv("train.csv")
data_copy = data.copy()
print(data.shape)
data.head(5)

(7395, 27)


,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [ ]:
#imp of features, correlation with target, remove numbers, check data uniformity (length same or not), nn for classification, how to decide features to be kept for model, define title body in stopwords, how to use text data/large text data, summarise boilerplate, detect outliers, normal/? distribution of data, class imbalance problem

In [3]:
data['label'].value_counts() 

1    3796
0    3599
Name: label, dtype: int64

#### Conclusion: the data does not have a class imbalance problem

In [4]:
#check missing values

data.isnull().sum().sort_values(ascending=False)

label                             0
frameTagRatio                     0
alchemy_category                  0
alchemy_category_score            0
avglinksize                       0
commonlinkratio_1                 0
commonlinkratio_2                 0
commonlinkratio_3                 0
commonlinkratio_4                 0
compression_ratio                 0
embed_ratio                       0
framebased                        0
hasDomainLink                     0
spelling_errors_ratio             0
html_ratio                        0
image_ratio                       0
is_news                           0
lengthyLinkDomain                 0
linkwordscore                     0
news_front_page                   0
non_markup_alphanum_characters    0
numberOfLinks                     0
numwords_in_url                   0
parametrizedLinkRatio             0
boilerplate                       0
dtype: int64

In [9]:
l=list(data['alchemy_category'])
l.count('?')

2342

In [24]:
smalltraindata = data.iloc[0:500,:]

In [25]:
def clean_data(data):
    data.drop(['url','urlid'],axis=1,inplace=True)
    stopwords = s.words('english')
    for i in range (0, data.shape[0]):
        words_to_remove = ['''"title"''','''"body"''','''"url"''']
        for w in words_to_remove:
            data['boilerplate'].loc[i] = (data['boilerplate'].loc[i]).replace(w, "")
        data['boilerplate'].loc[i] = re.sub(r'[^\w\s]',"", data['boilerplate'].loc[i])
        word_tokens = word_tokenize(data['boilerplate'].loc[i])
        data['boilerplate'].loc[i]=" ".join(filter(lambda x: x not in stopwords , word_tokens))
        data['boilerplate'].loc[i]= re.sub('\d',"",data['boilerplate'].loc[i])
        
    
    
clean_data (data)

In [ ]:
data.to_csv('C:/Users/Gunisha Chaturvedi/Dropbox/My PC (LAPTOP-1CT80JS5)/Documents/internships/smalltraindata.csv')

In [ ]:
smalltraindata = pd.read_csv('C:/Users/Gunisha Chaturvedi/Dropbox/My PC (LAPTOP-1CT80JS5)/Documents/internships/smalltraindata.csv')

### Check correlation of alchemy_category with label using chi-squared test

In [64]:
pd.crosstab(data['label'],data['alchemy_category'])

alchemy_category,?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,science_technology,sports,unknown,weather
label,,,,,,,,,,,,,,
0,1166,591,254,223,186,48,216,18,388,42,157,302,4,4
1,1176,350,626,73,157,28,290,13,841,30,132,78,2,0


In [74]:
# h0 = no relationship bw variables

cont_table = [[591,254,223,186,48,216,18,388,42,157,302,4,4], [350,626,73,157,28,290,13,841,30,132,78,2,0]] 
stat, p, dof, expected = chi2_contingency(cont_table) 
  
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)') 

p value is 3.857352828735912e-124
Dependent (reject H0)


In [55]:
print(len(data['boilerplate'][0]))
#data['label'][500]

4925


#### Observation: length of boiler plate code varies a lot 

In [11]:
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()
text = data['boilerplate'][0]
for word in text:
    word = lemm.lemmatize(word)

In [29]:
lemm.lemmatize("ran",pos ="v")

'run'

In [1]:
from keras.preprocessing.text import one_hot

sentence=['Fast cars are good',
          'Football is a famous sport',
          'Be happy Be positive']

vocab_size=1000

encoded_docs=[one_hot(d,vocab_size) for d in sentence]

In [3]:
encoded_docs

[[643, 917, 590, 806], [866, 599, 532, 150, 923], [28, 261, 28, 121]]

In [5]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding
import numpy as np

In [34]:
# embedding_length=5
# max_length=10

# encoded_docs=pad_sequences(encoded_docs,truncating='post',padding='post',maxlen=max_length)
# print(encoded_docs)

In [33]:
# model=Sequential()
# model.add(Embedding(vocab_size,embedding_length,input_length=max_length))
# model.compile('rmsprop','mse')
# model.summary()
# output=model.predict(encoded_docs)
# print(output.shape)
# print(output)

In [56]:
length=[]
for i in range (0, data.shape[0]):
    length.append(len(data['boilerplate'].loc[i]))

l = pd.Series(length)
sum(l)/len(l)

1847.754

In [44]:
#testdata = pd.read_csv("test.csv")
testdata = data_copy.iloc[0:70,:]
clean_data(testdata)

C:\Users\Gunisha Chaturvedi\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Gunisha Chaturvedi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Gunisha Chaturvedi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Gunisha Chaturvedi\Anaconda3\lib\si

In [73]:
#xtrain, ytrain = data['boilerplate'], data['label']
#xtest, ytest = testdata['boilerplate'], testdata['label']
#vocab_size, embed_size = 2000, 20
#model = tf.keras.Sequential()
#max_length= 1847
#from keras.preprocessing.text import one_hot
#encoded_docs_train=[one_hot(d,vocab_size) for d in xtrain]
#encoded_docs_test=[one_hot(d,vocab_size) for d in xtest]
#xtrain = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs_train, maxlen= max_length)
#xtest = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs_test, maxlen= max_length)
#model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_length= max_length))
#model.add(tf.keras.layers.LSTM (units=70, activation='tanh'))
#model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#model.summary()
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#model.fit(xtrain, ytrain, epochs=5, batch_size=128)

test_loss, test_acurracy = model.evaluate(xtest, ytest)
print("Test accuracy: {}".format(test_acurracy))

3/3 [==============================] - 0s 10ms/step - loss: 0.5249 - accuracy: 0.9286
Test accuracy: 0.9285714030265808


In [59]:
import numpy as np
import tensorflow as tf

"""Loading the Dataset"""

from tensorflow.keras.datasets import imdb

"""### **Data Preprocessing**"""

words=20000
max_length=100

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=words)

"""Padding the Text"""

# x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_length)

# x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_length)


'Padding the Text'

In [61]:
xtrain

0      IBM Sees Holographic Calls Air Breathing Batte...
1      The Fully Electronic Futuristic Starting Gun T...
2      Fruits Fight Flu fruits fight flu cold flu men...
3       Foolproof Tips Better Sleep There period life...
4      The  Coolest Jerseys You Didn Know Existed coo...
5      conveniencemedical genital herpes home phpGeni...
6      fashion lane American Wild Child Our favorite ...
7      insidershealth article racing recovery Racing ...
8      Valet The Handbook  Days  daysResolutions suck...
9      howsweeteats    cookies cream browniesCookies ...
10     Business Financial News Breaking US Internatio...
11     A Tip Cap The Greatest Iron Man Them All As su...
12      Foods That Trash Your Teeth There  different ...
13     thedailygreen print healthy eating eat safe Di...
14     phillyburbs blogs type kitchen french onion st...
15     Izabel Goulart Swimsuit Kikidoll  Sports Illus...
16     Liquid Mountaineering The Awesomer Link We pre...
17     nullThe annual Chap Olym